## COMPANY VALUATION.

Core Idea: DCF discounts projected free cash flows (FCF) to present value using WACC, then adds terminal value. Comps multiply metrics (e.g., EV/EBITDA) by peers' multiples for a market-based check. Scenarios (base/bull/bear) vary growth rates.
Here's a basic two-stage DCF + comps for a stock like AAPL, pulling data via yfinance.

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

def dcf_valuation(ticker, growth_rate=0.05, discount_rate=0.08, years=5, terminal_growth=0.03):
    """Simple two-stage DCF: Explicit forecast + terminal value."""
    stock = yf.Ticker(ticker)
    financials = stock.financials.iloc[:, 0]  # Latest year
    fcf = financials.get('Free Cash Flow', np.nan)  # Or compute: OCF - CapEx
    
    if np.isnan(fcf):
        fcf = stock.cashflow.loc['Total Cash From Operating Activities'].iloc[0] - \
              stock.cashflow.loc['Capital Expenditures'].iloc[0]
    
    # Stage 1: Explicit FCF projections
    fcfs = [fcf * (1 + growth_rate)**i for i in range(1, years + 1)]
    pv_fcfs = [fcf / (1 + discount_rate)**i for fcf, i in zip(fcfs, range(1, years + 1))]
    
    # Stage 2: Terminal value (Gordon Growth)
    terminal_fcf = fcfs[-1] * (1 + terminal_growth)
    terminal_value = terminal_fcf / (discount_rate - terminal_growth)
    pv_terminal = terminal_value / (1 + discount_rate)**years
    
    # Enterprise value + cash - debt = equity value
    cash = stock.balance_sheet.loc['Cash'].iloc[0]
    debt = stock.balance_sheet.loc['Short Long Term Debt'].sum()
    equity_value = sum(pv_fcfs) + pv_terminal + cash - debt
    shares = stock.info['sharesOutstanding']
    intrinsic_per_share = equity_value / shares
    
    return intrinsic_per_share

def comps_valuation(ticker, peers=['MSFT', 'GOOGL'], metric='EV/EBITDA'):
    """Comparable companies: Avg peer multiple * target metric."""
    stock = yf.Ticker(ticker)
    ev = stock.info['enterpriseValue']
    ebitda = stock.financials.loc['EBITDA'].iloc[0]
    target_multiple = ev / ebitda if not np.isnan(ebitda) else np.nan
    
    peer_multiples = []
    for peer in peers:
        p_stock = yf.Ticker(peer)
        p_ev = p_stock.info['enterpriseValue']
        p_ebitda = p_stock.financials.loc['EBITDA'].iloc[0]
        peer_multiples.append(p_ev / p_ebitda if not np.isnan(p_ebitda) else np.nan)
    
    avg_peer_multiple = np.mean(peer_multiples)
    comps_value = avg_peer_multiple * ebitda
    equity_value = comps_value + stock.balance_sheet.loc['Cash'].iloc[0] - \
                   stock.balance_sheet.loc['Short Long Term Debt'].sum()
    shares = stock.info['sharesOutstanding']
    comps_per_share = equity_value / shares
    
    return comps_per_share, avg_peer_multiple

# Example: Run scenarios
ticker = 'AAPL'
base_dcf = dcf_valuation(ticker)
bull_dcf = dcf_valuation(ticker, growth_rate=0.07)  # Bull: Higher growth
bear_dcf = dcf_valuation(ticker, growth_rate=0.03)  # Bear: Lower growth
comps_val, multiple = comps_valuation(ticker)

print(f"DCF Base: ${base_dcf:.2f}/share")
print(f"DCF Bull: ${bull_dcf:.2f}/share")
print(f"DCF Bear: ${bear_dcf:.2f}/share")
print(f"Comps Value: ${comps_val:.2f}/share (Avg Multiple: {multiple:.1f}x)")